In [1]:
import pandas as pd
import cv2
# %matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
import tensorflow as tf
# import tensorflow.compat.v1 as tf
from tensorflow import keras
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import os
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from collections import OrderedDict
# from tensorflow.python.ops import math_ops
import tensorflow.keras.backend as kb
# from kerastuner.tuners import RandomSearch
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe
from keras.utils import normalize



Using TensorFlow backend.


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  2


In [3]:
tf.test.is_gpu_available()


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
def Files_Load():
    train_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Train_Box/"
    test_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Test_Box/"
    box_train_txt = os.listdir(train_file)
    box_train_txt.sort()
    box_test_txt = os.listdir(test_file)
    box_test_txt.sort()
    
    loc_files_train, loc_files_test = [], []
    
    for txt in box_train_txt:
        loc_files_train.append(train_file + txt)
    for txt in box_test_txt:
        loc_files_test.append(test_file + txt)
    
    return loc_files_train, loc_files_test, box_train_txt, box_test_txt

# Faster Data Loader

In [5]:
def Boxes(loc_files, txt_names, time_steps, pad ='pre'):
    """
    loc_files: List that contains that has text files save
    txt_names: Txt file names. For visualization process
    time_step: Sequence length input
    pad: inputs 'pre' or 'post'
    
    x_person_box: Has bounding box locations
    y_person_box: Label for bounding box locations
    frame_person_id: Contains frame Number and person_Id of entire sequence, 
                     Last element is prediction frame. For visulization process
    video_file: Points to video file used. For visulization process
    """
    
    x_ppl_box, y_ppl_box, frame_ppl_id, video_file, abnormal = [], [], [], [],[]  #Has bounding box locations inside
    
    #For splitting process
    split_train_test = 0
    split = 0
    find_split = 0
    
    # Tells me how many in sequence was short.
    # Do I want to go back and count for train and test seperatly 
    short_len = 0
    
#     datadict = OrderedDict()
    datadict = {}
    
    for loc, txt_name in zip(loc_files, txt_names):
        data = pd.read_csv(loc, ' ' )
        # Note that person_box is 1 behind ID
        max_person = data['Person_ID'].max()
        for num in range(1,max_person+1):
            temp_box = data[data['Person_ID'] == num ]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()].values
            person_seq_len = len(temp_box)
            temp_frame_id = data[data['Person_ID'] == num ]['Frame_Number Person_ID'.split()].values
            abnormal_frame_ped = data[data['Person_ID'] == num]['anomaly'].values
            if person_seq_len > time_steps:
                for i in range(0, person_seq_len - time_steps):
                    temp_person_box = temp_box[i:(i+time_steps)]
                    temp_fr_person_id = temp_frame_id[i:(i+time_steps+1)]

                    x_ppl_box.append(temp_person_box)
                    y_ppl_box.append(temp_box[i+time_steps])
                    
                    assert temp_person_box.shape == (time_steps,4)
                    assert temp_fr_person_id.shape  == (time_steps+1,2), print(temp_fr_person_id.shape)
                    
                    frame_ppl_id.append(temp_fr_person_id)
                    
                    video_file.append(txt_name)      
                    abnormal.append(abnormal_frame_ped[i+time_steps]) #Finds if predicted frame is abnormal
                    
            elif person_seq_len == 1:
                # want it to skip loop
                continue
            elif person_seq_len <= time_steps:
                temp_person_box_unpad = temp_box
                temp_fr_person_id_unpad = temp_frame_id
                temp_person_box = pad_sequences(temp_person_box_unpad.T, maxlen = time_steps+1, padding = pad).T
                temp_fr_person_id = pad_sequences(temp_fr_person_id_unpad.T,  maxlen = time_steps+1, padding = pad).T
                
                assert temp_person_box.shape == (time_steps+1,4)
                assert temp_fr_person_id.shape  == (time_steps+1,2)
                
                x_ppl_box.append(temp_person_box[0:time_steps,:])
                y_ppl_box.append(temp_person_box[time_steps,:])
                
                frame_ppl_id.append(temp_fr_person_id[0:time_steps+1,:])
                
                video_file.append(txt_name)
                abnormal.append(abnormal_frame_ped[-1]) #Finds if predicted frame is abnormal

            else:
                print('error')

    
    datadict['x_ppl_box'] = np.array(x_ppl_box)
    datadict['y_ppl_box'] = np.array(y_ppl_box)
    datadict['frame_ppl_id'] = np.array(frame_ppl_id)
    datadict['video_file'] = np.array(video_file)
    datadict['abnormal'] = np.array(abnormal)

        
    return  datadict

# Video Plot Function

In [6]:
def videoPlot(y_ppl_box,y_pred, frame_ppl_id, video_file, vid_type ='test'):
    """
    Function is plotting the ground truth and predicted 
    
    y_ppl_box: Label for bounding box locations
    y_ppl_box_pred: Prediction from Model
    frame_person_id: Contains frame Number and person_Id of entire sequence, 
                     Last element is prediction frame. For visulization process
    video_file: Points to video file used. For visulization process
    """
    # Need a way to save images
    file = {}
    file['train'] = '/home/akanu/Dataset/Anomaly/Avenue_Dataset/training_videos/'
    file['test'] = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/testing_videos/"
    loc_videos = file[vid_type] + video_file[0][:2] + '.' + video_file[0][2:5]
    ###
    
    video_capture = cv2.VideoCapture(loc_videos)
    
    for i in range(-1, frame_ppl_id[0,-1,0] + 1):
        #Assune sequences are connected and don't skip cuz of occlusions 
        print(i)
        ret, frame = video_capture.read()
        if i == frame_ppl_id[0,-1,0]:
            pred_frame = frame.copy()
            cv2.rectangle(frame, (int(y_ppl_box[0,0]), int(y_ppl_box[0,1])), (int(y_ppl_box[0,2]), int(y_ppl_box[0,3])),(255,255,255), 2)
            cv2.putText(frame, str(frame_ppl_id[0,-1,1]),(int(y_ppl_box[0,0]), int(y_ppl_box[0,1])),0, 5e-3 * 200, (0,255,0),2)

            cv2.rectangle(pred_frame, (int(y_pred[0,0]), int(y_pred[0,1])), (int(y_pred[0,2]), int(y_pred[0,3])),(255,255,0), 2)
            cv2.putText(pred_frame, str(frame_ppl_id[0,-1,1]),(int(y_pred[0,0]), int(y_pred[0,1])),0, 5e-3 * 200, (0,255,0),2)
            
            cv2.imwrite('/home/akanu/git/deep_sort_yolov3/Images_saved/pred.jpg', pred_frame)
            cv2.imwrite("/home/akanu/git/deep_sort_yolov3/Images_saved/gt.jpg", frame)


# IOU Metric

In [7]:
def bb_intersection_over_union(y, x):
    xA = kb.max((x[:,0:1],y[:,0:1]), axis=0,keepdims=True)
    yA = kb.max((x[:,1:2],y[:,1:2]), axis=0,keepdims=True)
    xB = kb.min((x[:,2:3],y[:,2:3]), axis=0,keepdims=True)
    yB = kb.min((x[:,3:4],y[:,3:4]), axis=0,keepdims=True)

    interArea1 = kb.max((kb.zeros_like(xB), (xB-xA +1) ), axis=0, keepdims=True)
    interArea2 = kb.max((kb.zeros_like(xB), (yB-yA +1) ), axis=0, keepdims=True)
    interArea = interArea1*interArea2
    boxAArea = (x[:,2:3] - x[:,0:1] + 1) * (x[:,3:4] - x[:,1:2] + 1)
    boxBArea = (y[:,2:3] - y[:,0:1] + 1) * (y[:,3:4] - y[:,1:2] + 1)

    iou = interArea / (boxAArea + boxBArea - interArea)
    iou_mean = kb.mean(iou)
    return iou_mean

# Data Loading

In [8]:
len_frames = 20 #controls the amount of frames used to predict next bounding box

In [9]:
loc_files_train, loc_files_test, box_train_txt, box_test_txt = Files_Load()
start = time.time()
traindict = Boxes(loc_files_train, box_train_txt, len_frames, pad ='pre')
end = time.time()
print(end-start)

2.717153549194336


In [10]:
# normalize(traindict['x_ppl_box'], axis =1)

In [11]:
start = time.time()

testdict = Boxes(loc_files_test[3:4], box_test_txt[3:4], len_frames, pad ='pre')
end = time.time()
print(end -start)

0.2163527011871338


In [12]:
abnormal_index = np.nonzero(testdict['abnormal'])
normal_index = np.where(testdict['abnormal'] == 0)

In [13]:
abnormal_test_x = testdict['x_ppl_box'][abnormal_index]
abnormal_test_y = testdict['y_ppl_box'][abnormal_index]
test_x = testdict['x_ppl_box'][normal_index]
test_y = testdict['y_ppl_box'][normal_index]

In [14]:
max1 = traindict['x_ppl_box'].max()
min1 = traindict['x_ppl_box'].min()
xx = (traindict['x_ppl_box'] - min1)/(max1 - min1)
xx_test_abnorm = (abnormal_test_x - min1)/(max1-min1)
xx_test = (test_x - min1)/(max1-min1)

In [15]:
yy = (traindict['y_ppl_box'] - min1)/(max1-min1)
yy_test_abnorm = (abnormal_test_y - min1)/(max1-min1)
yy_test = (test_y - min1)/(max1-min1)

In [16]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32
test_size = 0.1
xx_train, xx_val,yy_train,yy_val = train_test_split(xx,yy, test_size = test_size)
train_univariate = tf.data.Dataset.from_tensor_slices((xx_train,yy_train))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
val_univariate = tf.data.Dataset.from_tensor_slices((xx_val,yy_val))
val_univariate = val_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Data Load Function

In [17]:
def data():
    
    def Files_Load():
        train_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Train_Box/"
        test_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Test_Box/"
        box_train_txt = os.listdir(train_file)
        box_train_txt.sort()
        box_test_txt = os.listdir(test_file)
        box_test_txt.sort()

        loc_files_train, loc_files_test = [], []

        for txt in box_train_txt:
            loc_files_train.append(train_file + txt)
        for txt in box_test_txt:
            loc_files_test.append(test_file + txt)

        return loc_files_train, loc_files_test, box_train_txt, box_test_txt

    
    def Boxes(loc_files, txt_names, time_steps, pad ='pre'):
        """
        loc_files: List that contains that has text files save
        txt_names: Txt file names. For visualization process
        time_step: Sequence length input
        pad: inputs 'pre' or 'post'

        x_person_box: Has bounding box locations
        y_person_box: Label for bounding box locations
        frame_person_id: Contains frame Number and person_Id of entire sequence, 
                         Last element is prediction frame. For visulization process
        video_file: Points to video file used. For visulization process
        """

        x_ppl_box, y_ppl_box, frame_ppl_id, video_file, abnormal = [], [], [], [],[]  #Has bounding box locations inside

        #For splitting process
        split_train_test = 0
        split = 0
        find_split = 0

        # Tells me how many in sequence was short.
        # Do I want to go back and count for train and test seperatly 
        short_len = 0

    #     datadict = OrderedDict()
        datadict = {}

        for loc, txt_name in zip(loc_files, txt_names):
            data = pd.read_csv(loc, ' ' )
            # Note that person_box is 1 behind ID
            max_person = data['Person_ID'].max()
            for num in range(1,max_person+1):
                temp_box = data[data['Person_ID'] == num ]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()].values
                person_seq_len = len(temp_box)
                temp_frame_id = data[data['Person_ID'] == num ]['Frame_Number Person_ID'.split()].values
                abnormal_frame_ped = data[data['Person_ID'] == num]['anomaly'].values
                if person_seq_len > time_steps:
                    for i in range(0, person_seq_len - time_steps):
                        temp_person_box = temp_box[i:(i+time_steps)]
                        temp_fr_person_id = temp_frame_id[i:(i+time_steps+1)]

                        x_ppl_box.append(temp_person_box)
                        y_ppl_box.append(temp_box[i+time_steps])

                        assert temp_person_box.shape == (time_steps,4)
                        assert temp_fr_person_id.shape  == (time_steps+1,2), print(temp_fr_person_id.shape)

                        frame_ppl_id.append(temp_fr_person_id)

                        video_file.append(txt_name)      
                        abnormal.append(abnormal_frame_ped[i+time_steps]) #Finds if predicted frame is abnormal

                elif person_seq_len == 1:
                    # want it to skip loop
                    continue
                elif person_seq_len <= time_steps:
                    temp_person_box_unpad = temp_box
                    temp_fr_person_id_unpad = temp_frame_id
                    temp_person_box = pad_sequences(temp_person_box_unpad.T, maxlen = time_steps+1, padding = pad).T
                    temp_fr_person_id = pad_sequences(temp_fr_person_id_unpad.T,  maxlen = time_steps+1, padding = pad).T

                    assert temp_person_box.shape == (time_steps+1,4)
                    assert temp_fr_person_id.shape  == (time_steps+1,2)

                    x_ppl_box.append(temp_person_box[0:time_steps,:])
                    y_ppl_box.append(temp_person_box[time_steps,:])

                    frame_ppl_id.append(temp_fr_person_id[0:time_steps+1,:])

                    video_file.append(txt_name)
                    abnormal.append(abnormal_frame_ped[-1]) #Finds if predicted frame is abnormal

                else:
                    print('error')


        datadict['x_ppl_box'] = np.array(x_ppl_box)
        datadict['y_ppl_box'] = np.array(y_ppl_box)
        datadict['frame_ppl_id'] = np.array(frame_ppl_id)
        datadict['video_file'] = np.array(video_file)
        datadict['abnormal'] = np.array(abnormal)


        return  datadict    
    
    

    len_frames =20
    
    loc_files_train, loc_files_test, box_train_txt, box_test_txt = Files_Load()
    traindict = Boxes(loc_files_train, box_train_txt, len_frames, pad ='pre')
    testdict = Boxes(loc_files_test[3:4], box_test_txt[3:4], len_frames, pad ='pre')
    

    max1 = traindict['x_ppl_box'].max()
    min1 = traindict['x_ppl_box'].min()
    x_train = (traindict['x_ppl_box'] - min1)/(max1 - min1)
    y_train = (traindict['y_ppl_box'] - min1)/(max1-min1)
    
    
    
    test_x = testdict['x_ppl_box']
    test_y = testdict['y_ppl_box']    
    x_test = (test_x - min1)/(max1-min1)
    y_test = (test_y - min1)/(max1-min1)


    return x_train,y_train,x_test,y_test

In [18]:
xx,yy,xx_test,yy_test = data()

# Create a function

In [19]:
# def build_lstm_model(n_hidden =2, n_neurons=1, learning_rate =5e-3, input_shape = xx.shape[-2:]):
#     lstm_model = keras.models.Sequential()
# #     lstm_model.add(keras.layers.InputLayer(input_shape=input_shape))
#     lstm_model.add(keras.layers.LSTM(n_neurons,return_sequences =True, input_shape=input_shape))
#     for layer in range(n_hidden):
#         lstm_model.add(keras.layers.LSTM({{choice([1, 2,3,4,5,6,7,8,9,10])}},return_sequences =True))
#     lstm_model.add(keras.layers.LSTM(4) )
#     lstm_model.add(keras.layers.Dense(4) )
#     opt = tf.keras.optimizers.Adam(learning_rate={{uniform(1e-, 1)}})
#     lstm_model.compile(loss='mse', optimizer=opt, metrics = bb_intersection_over_union)
#     return lstm_model
    

In [20]:
def create_model(x_train,y_train,x_test,y_test):
    def bb_intersection_over_union(y, x):
        xA = kb.max((x[:,0:1],y[:,0:1]), axis=0,keepdims=True)
        yA = kb.max((x[:,1:2],y[:,1:2]), axis=0,keepdims=True)
        xB = kb.min((x[:,2:3],y[:,2:3]), axis=0,keepdims=True)
        yB = kb.min((x[:,3:4],y[:,3:4]), axis=0,keepdims=True)

        interArea1 = kb.max((kb.zeros_like(xB), (xB-xA +1) ), axis=0, keepdims=True)
        interArea2 = kb.max((kb.zeros_like(xB), (yB-yA +1) ), axis=0, keepdims=True)
        interArea = interArea1*interArea2
        boxAArea = (x[:,2:3] - x[:,0:1] + 1) * (x[:,3:4] - x[:,1:2] + 1)
        boxBArea = (y[:,2:3] - y[:,0:1] + 1) * (y[:,3:4] - y[:,1:2] + 1)

        iou = interArea / (boxAArea + boxBArea - interArea)
        iou_mean = kb.mean(iou)
        return iou_mean
    
    
    
    
    lstm_model = keras.models.Sequential()
#     lstm_model.add(keras.layers.InputLayer(input_shape=x_train.shape[-2:]))
    lstm_model.add(keras.layers.LSTM({{choice([2,10])}},return_sequences =True, input_shape=x_train.shape[-2:]))
    n_hidden =5
    for layer in range(n_hidden):
        lstm_model.add(keras.layers.LSTM({{choice([2,10])}},return_sequences =True))
    lstm_model.add(keras.layers.LSTM(4) )
    lstm_model.add(keras.layers.Dense(4) )
    opt = tf.keras.optimizers.Adam(learning_rate={{choice([1e-7, 1e-5])}})
    lstm_model.compile(loss='mse', optimizer=opt, metrics = bb_intersection_over_union)
    lstm_history = lstm_model.fit(x_train, y_train, 
                                  batch_size= 32, 
                                  epochs=2,
                                  verbose=2,
                                  validation_split =0.1)
#     iou = np.amax(lstm_history.history['bb_intersection_over_union'])
    iou = np.amax(lstm_history.history['loss'])

    print('Best iou acc of epoch:', iou)
    return {'loss': -iou, 'status': STATUS_OK, 'lstm_model': lstm_model }

In [21]:
# keras.backend.clear_session()
# keras_reg = keras.wrappers.scikit_learn.KerasRegressor(lstm_model)

In [22]:
# keras_reg.fit(xx_train,yy_train,epochs=2, 
#               validation_data = (xx_val, yy_val),
#              callbacks =[keras.callbacks.EarlyStopping(patience=10)])

In [23]:
# x_train, y_train, x_test, y_test = data()

best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=1,
                                          trials=Trials(),
                                          notebook_name='Fine_Tuning_Neural_Network_Hyperparameters')
 

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import cv2
except:
    pass

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    import matplotlib
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.preprocessing.sequence import pad_sequences
except:
    pass

try:
    import os
except:
    pass

try:
    import time
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from collections import OrderedDict
except:
    pass

try:
    import tensorflow.keras.backend as kb
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
ex

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

job exception: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run GatherV2: Dst tensor is not initialized. [Op:GatherV2]


  0%|          | 0/1 [00:11<?, ?trial/s, best loss=?]


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run GatherV2: Dst tensor is not initialized. [Op:GatherV2]

In [33]:
def model1(x_train,y_train, x_test,y_test):
    model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=x_train.shape[-2:]),
    keras.layers.Dense({{choice([20,50,100])}})
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(loss="mse", optimizer=opt, metrics=['accuracy'])
    result = model.fit(x_train,y_train,
                      batch_size= 32,
                      epochs =2,
                      validation_split=0.1)
    validation_acc = np.amax(result.history['val_acc'])
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}





In [ ]:
s